# gurobiによる制約最適化を行う

In [5]:
import math
import random
import networkx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx
from gurobipy import *

## 変数
- 人iが避難施設kに避難した場合に１　バイナリー変数 x[i,k] ,x[j,l]

- 避難施設kとlの間に定義される伝染危険率 D[k,l]

- 人i,jでの伝染危険率 W[i,j]

## 定数

- max(jの感染率 – iの感染率,0)・iの死亡率 + max(iの感染率 – jの感染率,0)・ jの死亡率 
## 目的関数
-  W[i,j] D[k,l] (X[i]=k) (X[j]=l)の合計　の最小化

## 制約
- すべての人 iに対して，x[i,k]の施設kに対する合計 = 1   
- 施設kに対してx[i]=kの人に対する合計が施設kの容量以下

## 必要な集合
- 人の集合I
- 施設集合K
- 施設の容量
- 

In [12]:
I = [i for i in range(30)] #人の集合
q = {} 
for i in V:
    q[0] = 0
    q[i] = random.randint(10,20)
q

{0: 0,
 1: 20,
 2: 20,
 3: 20,
 4: 13,
 5: 18,
 6: 18,
 7: 12,
 8: 14,
 9: 10,
 10: 20,
 11: 14,
 12: 19,
 13: 15,
 14: 11,
 15: 13,
 16: 11,
 17: 15,
 18: 18,
 19: 18,
 20: 13,
 21: 13,
 22: 10,
 23: 10,
 24: 17,
 25: 12,
 26: 11,
 27: 16,
 28: 10,
 29: 12}

In [6]:
model = Model()

Using license file /Users/suzukinatsuki/gurobi.lic
Academic license - for non-commercial use only


In [ ]:
#変数
x = {}
for i in V:
    for j in V:
        for t in T:
            if j > i and i == V[0]:       # depot
                x[i,j,t] = model.addVar(ub=2, vtype="I", name="x(%s,%s,%s)"%(i,j,t))
            elif j > i:
                x[i,j,t] = model.addVar(ub=1, vtype="I", name="x(%s,%s,%s)"%(i,j,t))


model.update()
#for (i,j,t) in x:
#    print(x[i,j,t])

In [ ]:
#人iが避難施設kに避難した場合に1になる0-1変数 x[i,k] 
for i in I:
    for k in K:
        x[i,k] = model.addVar((ub=1, vtype="I", name="x(%s,%s)"%(i,k)))